In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import SVC

from bs4 import BeautifulSoup
import re

sns.set_style('darkgrid')

In [6]:
df = pd.read_csv("./data/IMDB Dataset.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
df.sample(5)

,review,sentiment
9876,This movie is of interest to the fans of the f...,negative
36698,It is an interesting exercise to witness the e...,positive
42772,Great 1980s Comic Strip comedy set in the Sout...,positive
10307,I keep waiting for Peter Fonda to start acting...,negative
33237,Richard Donner shows off his liberal credentia...,negative


In [10]:
df.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [11]:
df_pos = df[df['sentiment']=='positive'][:5000]
df_neg = df[df['sentiment']=='negative'][:5000]

df_reviews = pd.concat([df_pos, df_neg ])

In [12]:
from sklearn.model_selection import train_test_split


In [13]:
train,test = train_test_split(df_reviews,test_size =0.33,random_state=42)

In [14]:
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

In [15]:
train_y.value_counts()

sentiment
negative    3378
positive    3322
Name: count, dtype: int64

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)
test_x_vector = tfidf.transform(test_x)

In [18]:
train_x.shape

(6700,)

In [19]:
train_x_vector.shape

(6700, 44107)

In [20]:
type(train_x_vector)

scipy.sparse._csr.csr_matrix

In [26]:
primera_resenia = pd.DataFrame.sparse.from_spmatrix(train_x_vector,
                                  index=train_x.index,
                                  columns=tfidf.get_feature_names_out()).iloc[0]

In [27]:
primera_resenia

00           0
000          0
007          0
00am         0
00s          0
            ..
ísnt         0
île          0
önsjön       0
über         0
überwoman    0
Name: 6746, Length: 44107, dtype: Sparse[float64, 0]

In [28]:
train_x.iloc[0]

"I happened to rent this movie with my sister in hopes of watching a great entertaining movie, that was humorous, however my expectations were let down. This movie was beyond disgusting and revolting for a PG-13 movie, this should have been rated R for the many mature references that went on in this movie. I wouldn't recommend allowing a 13 year old teen see this.<br /><br />Even if no one under the age of 17 is watching this movie, beware of a truly stupid movie, there's no humor in the movie, just a bunch of disgusting sexual references including a small touch of pedophilia, something that shouldn't even be joked about. <br /><br />I would like to know what happened to PG-13 movies, that were actually safe for actual a 13 year old? This is beyond a deplorable movie and should be re-rated."

In [29]:
primera_resenia[primera_resenia != 0]

13               0.45849
17               0.12824
actual          0.091601
actually        0.061461
age             0.088765
allowing         0.12824
beware          0.143046
br              0.124945
bunch           0.093128
deplorable      0.168137
disgusting      0.237945
entertaining    0.081088
expectations    0.103149
great           0.048868
happened        0.176853
hopes           0.114028
humor           0.084465
humorous        0.116516
including       0.087603
joked           0.168137
just            0.037675
know            0.053984
let              0.07195
like            0.036454
mature          0.126021
movie           0.276309
movies            0.0522
old             0.123513
pedophilia      0.172712
pg              0.260154
rated           0.206643
recommend       0.076198
references      0.226901
rent            0.093234
revolting       0.151971
safe            0.119732
sexual          0.098677
shouldn         0.108203
sister          0.095008
small           0.078916


In [30]:
train_x.iloc[0]

"I happened to rent this movie with my sister in hopes of watching a great entertaining movie, that was humorous, however my expectations were let down. This movie was beyond disgusting and revolting for a PG-13 movie, this should have been rated R for the many mature references that went on in this movie. I wouldn't recommend allowing a 13 year old teen see this.<br /><br />Even if no one under the age of 17 is watching this movie, beware of a truly stupid movie, there's no humor in the movie, just a bunch of disgusting sexual references including a small touch of pedophilia, something that shouldn't even be joked about. <br /><br />I would like to know what happened to PG-13 movies, that were actually safe for actual a 13 year old? This is beyond a deplorable movie and should be re-rated."

In [31]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [32]:
print(svc.predict(tfidf.transform(['A good movie'])))
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['I did not like this movie at all I gave this movie away'])))

['positive']
['positive']
['negative']


In [33]:
print(svc.score(test_x_vector, test_y))

0.8706060606060606


In [34]:
from sklearn.metrics import f1_score

f1_score(test_y,svc.predict(test_x_vector),
          labels = ['positive','negative'],average=None)

array([0.87400413, 0.86701962])

In [35]:
from sklearn.metrics import classification_report

print(classification_report(test_y,
                            svc.predict(test_x_vector),
                            labels = ['positive','negative']))

              precision    recall  f1-score   support

    positive       0.87      0.88      0.87      1678
    negative       0.88      0.86      0.87      1622

    accuracy                           0.87      3300
   macro avg       0.87      0.87      0.87      3300
weighted avg       0.87      0.87      0.87      3300



In [36]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(test_y,
                           svc.predict(test_x_vector),
                           labels = ['positive', 'negative'])
conf_mat

array([[1481,  197],
       [ 230, 1392]])

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Entrenar el modelo
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(train_x_vector, train_y)

# Predecir y evaluar
lr_pred = lr.predict(test_x_vector)
lr_score = lr.score(test_x_vector, test_y)

print("### 🥇 Regresión Logística ###")
print(f"Precisión (Score): {lr_score:.4f}")
print("\nInforme de Clasificación:")
print(classification_report(test_y, lr_pred, labels=['positive', 'negative']))

# Matriz de Confusión
lr_conf_mat = confusion_matrix(test_y, lr_pred, labels=['positive', 'negative'])
print("Matriz de Confusión:\n", lr_conf_mat)

### 🥇 Regresión Logística ###
Precisión (Score): 0.8718

Informe de Clasificación:
              precision    recall  f1-score   support

    positive       0.87      0.88      0.87      1678
    negative       0.88      0.86      0.87      1622

    accuracy                           0.87      3300
   macro avg       0.87      0.87      0.87      3300
weighted avg       0.87      0.87      0.87      3300

Matriz de Confusión:
 [[1479  199]
 [ 224 1398]]
